# A/B Test Analysis
We're going to conduct an Independent Samples T-test to analyse our A/B test. An Indepdent Samples T-test compares the differences between two means of two different samples

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats

Export your Qualtrics results to a .csv file and save it to you github repository. Import your .csv file, inspect it, and clean it where neccesary.

In [2]:
# If on Github, load your data
df_A = pd.read_csv(r"C:\Users\maxme\OneDrive\Documenten\GitHub\2022-23c-1fcmgt-reg-ai-01-MaxMeiners\Assignments\AB-test_HCAI_Version_A.csv")
df_B = pd.read_csv(r"C:\Users\maxme\OneDrive\Documenten\GitHub\2022-23c-1fcmgt-reg-ai-01-MaxMeiners\Assignments\AB-test_HCAI_Version_B.csv")

# EDA A
df_A.info() # Is your data in the right format?
df_A.head() # Quick EDA. No? Clean it, you only want the rows and collumns containing likert-score data.

# EDA B
df_A.info() # Is your data in the right format?
df_A.head() # Quick EDA. No? Clean it, you only want the rows and collumns containing likert-score data.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   StartDate              2 non-null      object
 1   EndDate                2 non-null      object
 2   Status                 2 non-null      object
 3   IPAddress              2 non-null      object
 4   Progress               2 non-null      object
 5   Duration (in seconds)  2 non-null      object
 6   Finished               2 non-null      object
 7   RecordedDate           2 non-null      object
 8   ResponseId             2 non-null      object
 9   RecipientLastName      2 non-null      object
 10  RecipientFirstName     2 non-null      object
 11  RecipientEmail         2 non-null      object
 12  ExternalReference      2 non-null      object
 13  LocationLatitude       2 non-null      object
 14  LocationLongitude      2 non-null      object
 15  DistributionChannel    2 no

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,...,ExternalReference,LocationLatitude,LocationLongitude,DistributionChannel,UserLanguage,Q1_1,Q1_2,Q1_3,Q1_4,Q1_5
0,Start Date,End Date,Response Type,IP Address,Progress,Duration (in seconds),Finished,Recorded Date,Response ID,Recipient Last Name,...,External Data Reference,Location Latitude,Location Longitude,Distribution Channel,User Language,Please indicate to what extent the following s...,Please indicate to what extent the following s...,Please indicate to what extent the following s...,Please indicate to what extent the following s...,Please indicate to what extent the following s...
1,"{""ImportId"":""startDate"",""timeZone"":""America/De...","{""ImportId"":""endDate"",""timeZone"":""America/Denv...","{""ImportId"":""status""}","{""ImportId"":""ipAddress""}","{""ImportId"":""progress""}","{""ImportId"":""duration""}","{""ImportId"":""finished""}","{""ImportId"":""recordedDate"",""timeZone"":""America...","{""ImportId"":""_recordId""}","{""ImportId"":""recipientLastName""}",...,"{""ImportId"":""externalDataReference""}","{""ImportId"":""locationLatitude""}","{""ImportId"":""locationLongitude""}","{""ImportId"":""distributionChannel""}","{""ImportId"":""userLanguage""}","{""ImportId"":""QID1_1""}","{""ImportId"":""QID1_2""}","{""ImportId"":""QID1_3""}","{""ImportId"":""QID1_4""}","{""ImportId"":""QID1_5""}"


The rest we leave for tomorrow when we actually have our data. But if you are eager to play around a bit you can simply refresh the survey and fill in a couple of responses to create an A and a B version.

Now, let's start analysing our gather data! This block we won't dive into inferential statistics since it can get complex quite fast; we'll do that in Year 2, block B. For now, you just need to know that we need to test whether the data is normally distributed and whether the variances of both samples are equal. Otherwise, our statistical tests would not be valid and we can therefore not say that the results we're seeing are due to chance. What we are essentially going to statistically ascertain is whether there is a statistically significant different in the mean of a given variable for version A or B. 

In [3]:
# Create an array for each question you are going to compare which saves the sampling difference.
sampling_difference = df_A['SameQuestion_DifferentDF'].values - \
                      df_B['SameQuestion_DifferentDF'].values

# Then run the shaprio-wilk statistical test to check whether the data is normally distributed (for small samples)
normal = stats.shapiro(sampling_difference)

# Check whether the variance of both samples is equal
homogeneity = stats.levene(df_A['A'],
                           df_B['B'])

print(f"If the p-value is above 0.05, then the data is normally distrubted", normal, ". If the data is not normally distributed then you will have to run the bootstrapped version.")
print(f"If the p-value is above 0.05, then the groups have equal variances", homogeneity, ". If the variance aren't equal then you will have to run the bootstrapped version.")

KeyError: 'SameQuestion_DifferentDF'

Now, that dat is in the right format and we know the collumn names. Replace 'A' with the collumn name which holds your original baseline version; A. Replayce 'B' with the collumn name which holds the result of your improved version; B.

In [4]:
# Run Independent Samples T-test when assumptions are not violated.
results = stats.ttest_ind(df_A['A'],
                          df_B['B'])

# Print the results
print(f"The results are significant if the p-value is significant, which means => 0.05", 
results,
"\n", 
"If the results are significant, that means that the version are different enough to exclude chance for being the driver. So if you version has a higher/lower average score and is statistically significant, then it works better/worse. If the results are not significant then the changes don't have a real measureable effect so maybe it's no better or maybe the questions don't really measure the effect and you should consider rephrase or removing them.There's more to it but that for inferential statistics in year 2.")

KeyError: 'A'

In [22]:
# Run Bootstrapped Independent Samples T-test when assumptions are violated
rng = np.random.default_rng() # create random sampling

results = stats.ttest_ind(df_A['A'],
                          df_B['B'],
                          random_state = rng)

# Print the results
print(f"The results are significant if the p-value is significant, which means => 0.05", 
results,
"\n", 
"If the results are significant, that means that the version are different enough to exclude chance for being the driver. So if you version has a higher/lower average score and is statistically significant, then it works better/worse. If the results are not significant then the changes don't have a real measureable effect so maybe it's no better or maybe the questions don't really measure the effect and you should consider rephrase or removing them.There's more to it but that for inferential statistics in year 2.")

The results are significant if the p-value is significant, which means => 0.05 Ttest_indResult(statistic=3.0669836819036274, pvalue=0.002412277478078891) 
 In truth, inferential statistics is more complicated then that but for our purposes we'll stick to this.


Great, that was our first t-test. Save the results to your learning log in the week 8 and interpret them there. Were they what you expected? What are you going to change to improve your design if neccesary. 